# 2. Heterogeneity Measurement

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from scipy.ndimage import distance_transform_edt as dst
import sys
sys.path.insert(0,r'../src')
from Vsi import Vsi, rock_type, read_image
import os
import tifffile
import glob

## Load the data

In [ ]:
img_list = glob.glob('../Data/*.ubc')
img_list

In [ ]:
# Reading Images
img_list = glob.glob('../Data/*.ubc')
castlegate, sandpack, gambier, beadpack  = [np.fromfile(img,dtype='uint8') for img in img_list]

# Reshaping Images
gambier = gambier.reshape((512,512,512))
castlegate = castlegate.reshape((512,512,512))
beadpack = beadpack.reshape((512,512,512))
sandpack = sandpack.reshape((512,512,512))

# Iverting the binary arrays (switching void and solid phases)
gambier = np.invert(gambier==True).astype('uint8')
castlegate = np.invert(castlegate==True).astype('uint8')
beadpack = np.invert(beadpack==True).astype('uint8')
sandpack = np.invert(sandpack==True).astype('uint8')

prepared_images = [castlegate, sandpack, gambier, beadpack]

### Scale-independent Variance

In [ ]:
res={}
for i, f_name in enumerate(img_list):
    im = prepared_images[i]
    # ----- get the relative radius-----------
    ds=dst(im[0]) # distance map of a slice, for fast computation. 
    mn_r1 = int(ds.max()) # maximum width of pores is used as minimum radius for moving windows
    mx_r1 = mn_r1+100
    #print('starting radius for '+f_name[:-4]+' is '+str(mn_r1)+' voxels')
    #------run the Vsi--------------
    vf = Vsi(im, no_radii=20,
         no_samples_per_radius=200,
         min_radius=mn_r1, max_radius=mx_r1)
    res[f_name[:-4]] = vf

Plot result

In [ ]:
colors = ['blue', 'red', 'darkorange', 'cyan']
plt.subplot(211)
for i, k in enumerate(res.keys()):
    plt.plot(res[k].variance, label=k, color=colors[i])
res[k].plot()

plt.subplot(212)
for i, k in enumerate(res.keys()):
    plt.plot(res[k].variance, label=k, color=colors[i])
res[k].plot(fill=1,legend=0)
plt.yscale('log')
plt.tight_layout()

In [ ]:
for k in res.keys():
    print(k,res[k].rock_type())

In [ ]:
# result for 1 sample
res[k].result()

In [ ]:
df = pd.DataFrame(columns=['Radii','Variance','Sample'])
for k in res.keys():
    df0 = res[k].result()
    df0['Sample']=k
    df = pd.concat([df,df0])
df